In [1]:
import os

DATA_DIR = "data" # This may need to be changed on different machines

# Make sure we're in the correct directory and make sure the data directory exists
if not os.path.exists(DATA_DIR):
    os.chdir("../..") # Move up two directories because we're in src/nb and the data directory/path should be in/start at the root directory 
    assert os.path.exists(DATA_DIR), f"ERROR: DATA_DIR={DATA_DIR} not found"  # If we still can't see the data directory something is wrong

from tqdm.notebook import tqdm
import numpy as np

import torch
from torch import nn
# get Dataset class
from src.lib.decoder import Decoder
from src.lib.paraphrase_model import Paraphraser
from src.lib.style_classifier import StyleEncoder
from src.lib.util import to_device
from transformers import GPT2LMHeadModel, AdamW, GPT2Tokenizer

In [2]:

    
    


# class InputBuilder:

#     def __init__(self, state_dict):
#         if state_dict is not None:
#             self.positional_embeds = state_dict["positional_embeds"] # 
#             self.token_embeds = state_dict["token_embeds"] #
#             return
        
#         paraphraser = Paraphraser()
#         # Just look up these weights
#         self.positional_embeds = paraphraser.model.transformer.wpe.weight
#         self.token_embeds = paraphraser.model.transformer.wte.weight

#         # put everything on the cpu
#         self.positional_embeds = self.positional_embeds.to("cpu").detach()
#         self.token_embeds = self.token_embeds.to("cpu").detach()

#         self.stye_encoder = StyleEncoder()

#         torch.cuda.empty_cache()



#     def build_input(self, semantic_sentence, stlye_sentence):

#         style_encoding = self.stye_encoder.get_style_vector([stlye_sentence])
#         style_encoding = style_encoding.to("cpu").detach()


        
#         style = self.df["label"][idx]
#         random_style_encoding_idx = np.random.choice(np.where(self.df["label"] == style)[0])
#         style_encoding = self.style_encodings[random_style_encoding_idx]

#         para_ids = self.para_ids[idx]
#         para_embed = self.token_embeds[para_ids]
#         para_attn_mask = self.para_attn[idx]

#         target_ids = self.target_ids[idx]
#         target_embeds = self.token_embeds[target_ids]
#         target_attn_mask = self.target_attn[idx].detach().clone()

#         # select a random non-padding text index
#         selected_idx = np.random.choice(np.where(target_attn_mask == 1)[0])
#         # set the text_token_attn_mask to 0 starting at the selected index
#         target_attn_mask[selected_idx:] = 0

#         para_length = torch.sum(para_attn_mask).item()
#         para_pos_ids = np.arange(0, len(para_ids))
#         para_pos = self.positional_embeds[para_pos_ids]

#         bos_pos_id = para_length + 1
#         bos_pos = self.positional_embeds[bos_pos_id]

#         target_pos_ids = np.arange(para_length + 2, para_length + 2 + len(target_ids))
#         target_pos = self.positional_embeds[target_pos_ids]

#         # attn_mask = torch.tensor([[1], para_attn_mask, [1], target_attn_mask])
#         attn_mask = torch.ones(2 + len(para_attn_mask) + len(target_attn_mask))
#         attn_mask[1:len(para_attn_mask)+1] = para_attn_mask
#         attn_mask[len(para_attn_mask)+2:] = target_attn_mask

#         label = target_ids[selected_idx]
#         label_idx = len(para_ids) + 2 + selected_idx

#         style_encoding = style_encoding.detach()
#         para_embed = para_embed.detach()
#         para_pos = para_pos.detach()
#         target_embeds = target_embeds.detach()
#         target_pos = target_pos.detach()
#         attn_mask = attn_mask.detach()
#         label = label.detach()
#         bos_pos = bos_pos.detach()

#         return (
#             style_encoding, # Style encoding form BERT style classification of the target sentence
#             (para_embed, para_pos), # Token embeddings from the paraphrased sentence with positional embeddings
#             bos_pos, # Positional embedding of the BOS token
#             (target_embeds, target_pos), # Token embeddings from the target sentence with positional embeddings
#             attn_mask # Attention mask for the entire sequence
#         ), (
#             label, # Token id of the token to be predicted - index in vocab
#             label_idx # Index in the sequence of the token to be predicted - index in the sequence
#         )
    

#     def save_state_dict(self, path):
#         state = {
#             "df": self.df,
#             "style_encodings": self.style_encodings,
#             "positional_embeds": self.positional_embeds,
#             "token_embeds": self.token_embeds,
#             "para_ids": self.para_ids,
#             "para_attn": self.para_attn,
#             "target_ids": self.target_ids,
#             "target_attn": self.target_attn
#         }
#         torch.save(state, path)
    

#     @classmethod
#     def from_state_dict(cls, path):
#         state = torch.load(path)
#         return cls(state_dict=state)

In [3]:

def load_decoder(state_dict_path):
    state_dict = torch.load(state_dict_path)
    for key in state_dict:
        state_dict[key] = state_dict[key].cpu()
    decoder = Decoder()
    decoder.load_state_dict(state_dict)
    return decoder
    

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
path = "training_results/decoder_0_0.0979/model.pth"
decoder = load_decoder(path)

Some weights of the model checkpoint at models/gpt2_large were not used when initializing GPT2LMHeadModel: ['transformer.extra_embedding_project.bias', 'transformer.extra_embedding_project.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
style_encoder = StyleEncoder()

In [ ]:
# style_encoding = style_encoder.get_style_vector([style_sentence]).squeeze(0)
# style_encoding.shape

In [ ]:
# positional_embeds = decoder.gpt2.transformer.wpe.weight
# token_embeds = decoder.gpt2.transformer.wte.weight

In [6]:
def build_input(semantic_sentence, style_sentence, style_encoder, decoder):
    positional_embeds = decoder.gpt2.transformer.wpe.weight
    token_embeds = decoder.gpt2.transformer.wte.weight

    style_encoding = style_encoder.get_style_vector([style_sentence]).squeeze(0) # (768)

    max_length = 50
    tokenized = decoder.tokenizer([semantic_sentence], return_tensors="pt", truncation=True, max_length=max_length)
    para_ids = tokenized["input_ids"] # (1, para_length)
    para_attn = tokenized["attention_mask"].squeeze(0) # (para_length)

    para_embeds = token_embeds[para_ids].squeeze(0).detach() # (para_length, 1280)
    para_pos = positional_embeds[np.arange(0, len(para_embeds))].detach() # (para_length, 1280)

    target_embeds = token_embeds[[decoder.tokenizer.bos_token_id, decoder.tokenizer.pad_token_id]].detach() # (2, 1280)
    target_pos = positional_embeds[[len(para_ids), len(para_ids)+1]].detach() # (2, 1280)

    target_attn = torch.tensor([1, 0])

    attn_mask = torch.ones(2 + len(para_attn) + len(target_attn)) # (2 + para_length + target_length)
    attn_mask[1:len(para_attn)+1] = para_attn
    attn_mask[len(para_attn)+2:] = target_attn # just one for the BOS token
    attn_mask = attn_mask

    bos_pos = positional_embeds[len(para_ids) + 1].detach()

    style_encoding = style_encoding.unsqueeze(0)
    para_embeds = para_embeds.unsqueeze(0)
    para_pos = para_pos.unsqueeze(0)
    bos_pos = bos_pos.unsqueeze(0)
    target_embeds = target_embeds.unsqueeze(0)
    target_pos = target_pos.unsqueeze(0)
    attn_mask = attn_mask.unsqueeze(0)

    x = (
        style_encoding,
        (para_embeds, para_pos),
        bos_pos,
        (target_embeds, target_pos),
        attn_mask.unsqueeze(0)
    )

    return x


def generate(x, decoder, truncate=False, max_length=50, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    x = to_device(x, device)
    decoder = decoder.to(device)
    
    # print sum of attn_mask
    

    generated_ids = []
    generated_logits = []
    with torch.no_grad():
        i = 0
        while i < max_length or not truncate:
            i += 1

            style_encoding, para, bos_pos, target, attn_mask = x
            target_embeds, target_pos = target

            print(attn_mask.sum())

            output = decoder(x)

            logits = output.logits[0, -1, :]
            token_id = logits.argmax()

            generated_logits.append(logits)


            # check if token_id is eos
            if token_id == decoder.tokenizer.eos_token_id:
                break

            # add generated id 
            generated_ids.append(token_id.item())
            next_embedding = decoder.gpt2.transformer.wte.weight[token_id]
            next_pos_embed = decoder.gpt2.transformer.wpe.weight[target_pos.shape[1] + 1]

            # update the target embedding
            target_embeds = torch.cat([target_embeds, next_embedding.unsqueeze(0).unsqueeze(0)], dim=1)
            # update the target position
            target_pos = torch.cat([target_pos, next_pos_embed.unsqueeze(0).unsqueeze(0)], dim=1)
            # update the attention mask
            attn_mask = torch.cat([attn_mask, torch.ones(1, 1, 1).to(device)], dim=2)

            # repackage x
            x = (
                style_encoding,
                para,
                bos_pos,
                (target_embeds, target_pos),
                attn_mask
            )
        
    return generated_ids, generated_logits



In [7]:
semantic_sentence = "Hello, how are you?"
style_sentence = "The all-seeing sun Ne'er saw her match since first the world begun."

In [8]:
input_sequence = build_input(semantic_sentence, style_sentence, style_encoder, decoder)
tokens, logits = generate(input_sequence, decoder, device=device, truncate=True)

tensor(9., device='cuda:0')
tensor(10., device='cuda:0')
tensor(11., device='cuda:0')
tensor(12., device='cuda:0')
tensor(13., device='cuda:0')
tensor(14., device='cuda:0')
tensor(15., device='cuda:0')
tensor(16., device='cuda:0')
tensor(17., device='cuda:0')
tensor(18., device='cuda:0')
tensor(19., device='cuda:0')
tensor(20., device='cuda:0')
tensor(21., device='cuda:0')
tensor(22., device='cuda:0')
tensor(23., device='cuda:0')
tensor(24., device='cuda:0')
tensor(25., device='cuda:0')
tensor(26., device='cuda:0')
tensor(27., device='cuda:0')
tensor(28., device='cuda:0')
tensor(29., device='cuda:0')
tensor(30., device='cuda:0')
tensor(31., device='cuda:0')
tensor(32., device='cuda:0')
tensor(33., device='cuda:0')
tensor(34., device='cuda:0')
tensor(35., device='cuda:0')
tensor(36., device='cuda:0')
tensor(37., device='cuda:0')
tensor(38., device='cuda:0')
tensor(39., device='cuda:0')
tensor(40., device='cuda:0')
tensor(41., device='cuda:0')
tensor(42., device='cuda:0')
tensor(43., dev

In [9]:
print([logits[0].argmax()])
print([logits[1].argmax()])
print([logits[2].argmax()])
print([logits[3].argmax()])
print([logits[4].argmax()])

[tensor(15365, device='cuda:0')]
[tensor(15365, device='cuda:0')]
[tensor(15365, device='cuda:0')]
[tensor(15365, device='cuda:0')]
[tensor(15365, device='cuda:0')]


In [10]:
# decode the tokens
text = decoder.tokenizer.decode(tokens)
text

'ENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTSENTS'

In [ ]:
max_length = 50
tokenized = decoder.tokenizer([semantic_sentence], return_tensors="pt", truncation=True, max_length=max_length)
para_ids = tokenized["input_ids"]
para_attn = tokenized["attention_mask"].squeeze(0)
print(para_ids.shape)
print(para_attn.shape)

In [ ]:
para_embeds = token_embeds[para_ids].squeeze(0).detach()
para_pos = positional_embeds[np.arange(0, len(para_embeds))].detach()
# para_embeds += para_pos
print(para_embeds.shape)
print(para_pos.shape)

In [ ]:
target_embeds = token_embeds[decoder.tokenizer.bos_token_id].unsqueeze(0).detach()
target_pos = positional_embeds[len(para_ids)].unsqueeze(0).detach()
# target_embeds += target_pos
print(target_pos.shape)
print(target_embeds.shape)

In [ ]:
target_attn = torch.tensor([1])

attn_mask = torch.ones(2 + len(para_attn) + len(target_attn))
attn_mask[1:len(para_attn)+1] = para_attn
attn_mask[len(para_attn)+2:] = target_attn # just one for the BOS token
attn_mask = attn_mask
print(attn_mask.shape)


In [ ]:
bos_pos = positional_embeds[len(para_ids) + 1].detach()
print(bos_pos.shape)

In [ ]:
print("style_encoding", style_encoding.shape)
print("para_embeds", para_embeds.shape)
print("para_pos", para_pos.shape)
print("bos_pos", bos_pos.shape)
print("target_embeds", target_embeds.shape)
print("target_pos", target_pos.shape)
print("attn_mask", attn_mask.shape)

# unsqueeze to add batch dimension
style_encoding = style_encoding.unsqueeze(0)
para_embeds = para_embeds.unsqueeze(0)
para_pos = para_pos.unsqueeze(0)
bos_pos = bos_pos.unsqueeze(0)
target_embeds = target_embeds.unsqueeze(0)
target_pos = target_pos.unsqueeze(0)
attn_mask = attn_mask.unsqueeze(0)

In [ ]:
x = (
    style_encoding,
    (para_embeds, para_pos),
    bos_pos,
    (target_embeds, target_pos),
    attn_mask.unsqueeze(0)
)


In [ ]:
x = to_device(x, "cuda")
decoder = decoder.to("cuda")

In [ ]:
output = decoder(x)

In [ ]:
token_id = output.logits.argmax().item()